# Lab 1 - Michelson Interferometer
#### Jan Tache, Javier Velarde

## Objectives

- To observe and manipulate intereference patterns with different light sources
- To use the interfence patterns to determine information about the wavelengths in the light sources and the differences in the beam paths
- To use Fourier transform spectroscopy to determine the wavelengths in the different light sources

## Day 1 - 2018/01/16

Mount sodium lamp and white light lamp on a post with the double ended 90 degree offset clamp.

lower right knob is horizontal mirror alignment
upper left knob is vertical mirror alignment

Issues with APT User program
 - Would not display anything
 - Checked USB cable, connection was fine
 - Checked motor power - it was on
 - Rebooting the PC worked

Note: don't touch the table at all once MATLAB script is running. Any small movements will perturb the interferometer distances slightly. Even walking too hard will do so.

Method to get close to ZPL:

1. Use a metre stick and place the mirror on the stage and the 2nd mirror at as close to equal distance from the beam splitter as possible.
    - Measure from the closest face of the beam splitter to the front of the mirror
1. Use HeNe laser to align mirrors initially
1. check that on the outer surface of the beam splitter cube, the two points from one of the split beams were on top of each other
    - Do this individually for each mirror
1. Then align a piece of paper with a hole in it such that one of the laser beams passes through the hole
1. Now put a piece of paper in front of the camera, and adjust the mirrors slightly until the light shows nice interference rings
1. Place the diffusing lens in front of the HeNe laser and align it so that the beam still passes through the hole in the paper.
1. Adjust the stage using User(INSERT PROPER NAME HERE) program to get one large circle on the camera


Found somewhere close to ZPL at 1.4 unit on stage adjustment the program with HeNe laser.
![](images_2018-01-16/2018-01-16-zpl_with_hene.png)

After finding the ZPL, we switched to the sodium lamp:

1. turn off HeNe laser
1. swap the CCD filter to a camera lens
1. position the sodium lamp behind a light diffuser
1. turn on the sodium lamp - it takes a while to warm up and reach full intensity
1. adjust the aperture on the camera lens in order to be able to see the very faint interference pattern

Here is an example of an interference pattern that we found on the sodium lamp:

![](images_2018-01-16/2018-01-16-circles_moved_mirror_centre.png)

## Section 5.3 Finding the Fringes

Here are some different fringe patterns

1. ![](images_2018-01-16/2018-01-16-circles_moved_mirror_centre.png)

After aligning with the HeNe laser, we adjusted the stage to get maximum visibility with the mirrors centered
    
    
2. ![](images_2018-01-16/2018-01-16-sodium-maybe-zpl.png)

To produce this one, we adjusted the stage slightly from the previous to get desctructive inteference in the middle. Since the inner circle is so large, we assume that this is close to the ZPL.
    
    
3. ![](images_2018-01-16/2018-01-16-vertical_not_centered.png)

To produce vertical lines, we moved one of the mirrors using the lower knob to reflect slightly to the right


4. ![](images_2018-01-16/2018-01-16-vertical_narrower.png)

To produce thinner vertical lines, we moved the mirror from before to reflect even further to the right


5. ![](images_2018-01-16/2018-01-16-narrower_off_centre.png)

to produce horizontal curved lines, we did a similar procedure to 3 and 4, but used the upper left knob to reflect the image upwards


6. ![](images_2018-01-16/2018-01-16-even-narrower_off_centre.png)

to produce narrower curved lines, we did a similar procedure to 3 and 4, but used the upper left knob to move the image even further upwards


7. ![](images_2018-01-16/2018-01-16-sodium_part_circle.png)

To produce diagonal curved lines, we combined movements of both knobs.

8. ![](images_2018-01-16/2018-01-16-sodium_zpl_maybe_v2.png)

To produce a barely visible fringe pattern, we moved the stage so that the beams  differed by many wavelengths.



Answering the questions:

1. Justify why this is a good model for the interferometer you are using in the lab.

2. Write down the electric field amplitudes ($U_1$ and $U_2$) from two spherical wave sources and derive an expression for the total intensity $I = |U_1 +U_2|^2$ as seen on the observation screen.

3. Using this expression, describe how the intensity on axis varies with d.

4. Describe what happens to the interference pattern if you change d along the z-axis. Why might knowing this information be useful in this lab?



